# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation. To run this example Julia's `PyCall` package
needs to be able to find an installation of `pymatgen`.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

┌ Warning: `vendor()` is deprecated, use `BLAS.get_config()` and inspect the output instead
│   caller = npyinitialize() at numpy.jl:67
└ @ PyCall /home/runner/.julia/packages/PyCall/L0fLP/src/numpy.jl:67


Setup an LDA model and discretize using
a single k-point and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=(1, 1, 1))

PlaneWaveBasis discretization:
    Ecut                 : 5.0 Ha
    fft_size             : (32, 32, 32), 32768 total points
    kgrid type           : Monkhorst-Pack
    kgrid                : [1, 1, 1]
    num. irred. kpoints  : 1

    Discretized Model(lda_xc_teter93, 3D):
        lattice (in Bohr)    : [0         , 10.2631   , 10.2631   ]
                               [10.2631   , 0         , 10.2631   ]
                               [10.2631   , 10.2631   , 0         ]
        unit cell volume     : 2162.1 Bohr³
    
        atoms                : Si₁₆
        atom potentials      : ElementPsp(Si, psp="hgh/lda/si-q4")
    
        num. electrons       : 64
        spin polarization    : none
        temperature          : 0 Ha
    
        terms                : Kinetic()
                               AtomicLocal()
                               AtomicNonlocal()
                               Ewald()
                               PspCorrection()
                               

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.118004e+02     1.751060e+00
 * time: 0.4772989749908447
     1     1.057568e+01     8.923173e-01
 * time: 1.297929048538208
     2    -1.155901e+01     1.071310e+00
 * time: 1.3908109664916992
     3    -3.389846e+01     8.223062e-01
 * time: 1.5510990619659424
     4    -4.725541e+01     5.865324e-01
 * time: 1.656383991241455
     5    -5.663616e+01     2.140275e-01
 * time: 1.7636258602142334
     6    -5.959785e+01     1.648513e-01
 * time: 1.8566110134124756
     7    -6.080999e+01     5.535659e-02
 * time: 1.969588041305542
     8    -6.132293e+01     4.990539e-02
 * time: 2.048318862915039
     9    -6.160761e+01     3.019375e-02
 * time: 2.1292409896850586
    10    -6.179415e+01     3.046985e-02
 * time: 2.2091128826141357
    11    -6.195440e+01     2.152971e-02
 * time: 2.2874138355255127
    12    -6.205308e+01     1.773558e-02
 * time: 2.367421865463257
    13    -6.209686e+01     1.591552e-02
 * time: 2.463764905929565

In [5]:
scfres.energies

Energy breakdown (in Ha):
    Kinetic             25.7671073
    AtomicLocal         -18.8557695
    AtomicNonlocal      14.8522654
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485378 
    Xc                  -19.3336823

    total               -62.261666460449